In [59]:
import os
import pickle
import glob
from os.path import join, exists
from os import listdir
from pathlib import Path

import numpy as np
import tensorflow as tf
import pandas as pd
import scipy.stats
from matplotlib import pyplot as plt
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
%matplotlib notebook

In [160]:
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'TeX Gyre Heros'
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['text.usetex'] = False
# import matplotlib.font_manager
# matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')

In [129]:
MS_THESIS_OUT_DIR = "/home/carlo/KTH/thesis/ms-thesis/out/"
DDPG_FORK_OUT_DIR = "/home/carlo/KTH/thesis/misc/hindsight-experience-replay/out/"
CACHE_DIR = "/home/carlo/KTH/thesis/ms-thesis/out/results_viz_cache/"
FIGURES_DIR = "/home/carlo/KTH/thesis/ms-thesis/figures/"
os.makedirs(CACHE_DIR, exist_ok=True)

REMOTE_HOME = "/run/user/1000/gvfs/sftp:host=mordor.csc.kth.se,port=2222,user=carlora/home/carlora/"
REMOTE_MS_THESIS_OUT_DIR = f"{REMOTE_HOME}/thesis/repo/out/"
REMOTE_DDPG_FORK_OUT_DIR = f"{REMOTE_HOME}/thesis/exp/hindsight-experience-replay/out"

In [126]:
def read_tensorboard_log(path):
    # Loading too much data is slow...
    tf_size_guidance = {
        'distributions': 1,
        'images': 0,
        'scalars': 1000,
        'histograms': 1,
        'tensors': 1000,
    }
    event_acc = EventAccumulator(path, tf_size_guidance)
    event_acc.Reload()
    return event_acc

def tb_scalar_to_numpy(event_acc, tag_name):
    scalar = event_acc.Scalars(tag_name)
    return np.array([x[2] for x in scalar])

def data_from_tensorboard_log(path):
    tf.logging.set_verbosity(tf.logging.ERROR)
    event_acc = read_tensorboard_log(path)
    scalar_tags = event_acc.Tags()['scalars']
    results = {}
    for tag in scalar_tags:
        results[tag] = tb_scalar_to_numpy(event_acc, tag)
    return results

def data_from_tensorboard_dir(exp_dir):
    results = {}
    for sub_dir in listdir(exp_dir):
        results[sub_dir] = data_from_tensorboard_log(join(exp_dir, sub_dir))
    return results

def cache_result(unique_name, data):
    with open(f'{CACHE_DIR}/{unique_name}.pkl', 'wb') as f:
        pickle.dump(data, f)
    globals()[unique_name] = data
    return data

def get_cached_result(unique_name):
    if unique_name in globals():
        return globals()[unique_name]
    else:
        p = Path(f'{CACHE_DIR}/{unique_name}.pkl')
        if not p.exists():
            return None
        with open(p, 'rb') as f:
            data = pickle.load(f)
        globals()[unique_name] = data
        return data

In [80]:
def read_sb_ppo2_experiment_data(exp_name, remote=False):
    if remote:
        exp_dir = f'{REMOTE_MS_THESIS_OUT_DIR}/ppo2/{exp_name}'
    else:
        exp_dir = f'{MS_THESIS_OUT_DIR}/ppo2/{exp_name}'
    results = {}
    for sub_dir in listdir(exp_dir):
        sub_dir_p = join(exp_dir, sub_dir)
        for tb_dir in listdir(sub_dir_p):
            if 'tb' not in tb_dir:
                continue
            results[sub_dir] = data_from_tensorboard_log(join(sub_dir_p, tb_dir))
    return results

In [30]:
# read_sb_ppo2_experiment_data('fetch_stepped_rew_v3')

In [81]:
def read_her_torch_experiment_data(exp_name, remote=False):
    if remote:
        exp_dir = f'{REMOTE_DDPG_FORK_OUT_DIR}/her_torch/{exp_name}'
    else:
        exp_dir = f'{DDPG_FORK_OUT_DIR}/her_torch/{exp_name}'
    csvs = glob.glob(f'{exp_dir}/*/progress.csv')
    results = {}
    for c in csvs:
        c = Path(c)
        results[c.parent.name] = pd.read_csv(c)
    return results

In [76]:
# read_her_torch_experiment_data('fetch_weighted_her/baseline')

# Preliminary Experiments

### Fetch (sparse reward) with DDPG+HER

In [177]:
def _plot():
    from datetime import timedelta
    import matplotlib.ticker as ticker
    
    result_id = '_fetch_sparse_ddpg_her_baseline_new_local_res'
    data = get_cached_result(result_id)
    if data is None:
        data = read_her_torch_experiment_data('fetch_weighted_her/baseline_new')
        cache_result(result_id, data)
    
    df = list(data.values())[0]
    success_rate = df['test_success_rate'].values
    steps_per_epoch = df['env_steps'][0]

    fig = plt.figure(figsize=(8,4))
    ax1 = fig.gca()
    ax1.plot(df['training_iteration'].values, success_rate)
    ax1.set_ylabel('Success rate')
    ax1.set_xlabel('Epoch ({} env. steps each)'.format(steps_per_epoch))

    ax2 = ax1.twiny()
    ax2.plot(df['total_time'].values, success_rate).pop(0).remove()

    def _format_date(x, pos=None):
        return str(timedelta(seconds=x))

    ax2.xaxis.set_major_formatter(ticker.FuncFormatter(_format_date))
    for tick in ax2.get_xticklabels():
        tick.set_rotation(45)
        
    ax1.grid()

_plot()
# plt.show()
plt.savefig(f'{FIGURES_DIR}/fetch_sparse_her_torch.pdf', bbox_inches='tight')

'texgyreheros-regular.otf' can not be subsetted into a Type 3 font. The entire font will be embedded in the output.
'texgyreheros-regular.otf' can not be subsetted into a Type 3 font. The entire font will be embedded in the output.


### Fetch (stepped reward) with SB PPO2

In [188]:
def _plot():
    from datetime import timedelta
    import matplotlib.ticker as ticker

    result_id = '_fetch_stepped_rew_v3_ppo2_remote_res'
    data = get_cached_result(result_id)
    if data is None:
        data = read_sb_ppo2_experiment_data('fetch_stepped_rew_v3', remote=True)
        cache_result(result_id, data)

    data_dict = list(data.values())[0]
    success_rate = data_dict['custom/original_success_rate']
    steps_per_epoch = '?' # data_dict['env_steps'][0]

    fig = plt.figure(figsize=(8,4))
    ax1 = fig.gca()
    ax1.plot(success_rate)
    ax1.set_ylabel('Success rate')
    ax1.set_xlabel('Epoch ({} env. steps each)'.format(steps_per_epoch))

#     ax2 = ax1.twiny()
#     ax2.plot(df['total_time'].values, success_rate).pop(0).remove()

#     def _format_date(x, pos=None):
#         return str(timedelta(seconds=x))

#     ax2.xaxis.set_major_formatter(ticker.FuncFormatter(_format_date))
#     for tick in ax2.get_xticklabels():
#         tick.set_rotation(45)

    ax1.grid()

_plot()
# plt.show()
plt.savefig(f'{FIGURES_DIR}/fetch_stepped_sb_ppo2.pdf', bbox_inches='tight')

'texgyreheros-regular.otf' can not be subsetted into a Type 3 font. The entire font will be embedded in the output.
'texgyreheros-regular.otf' can not be subsetted into a Type 3 font. The entire font will be embedded in the output.


In [89]:
list(_fetch_stepped_rew_v3_ppo2_remote_res.values())[0].keys()

dict_keys(['loss/entropy_loss', 'loss/policy_gradient_loss', 'loss/value_function_loss', 'loss/approximate_kullback-leiber', 'loss/clip_factor', 'loss/loss', 'input_info/discounted_rewards', 'input_info/learning_rate', 'input_info/advantage', 'input_info/clip_range', 'input_info/old_neglog_action_probabilty', 'input_info/old_value_pred', 'episode_reward', 'custom/original_episode_reward', 'custom/original_success_rate'])

## YuMi Constrained (v2) DDPG+HER

In [184]:
def _plot():
    from datetime import timedelta
    import matplotlib.ticker as ticker
    
    result_id = '_yumi_constr_v2_sparse_ddpg_her_local_res'
    data = get_cached_result(result_id)
    if data is None:
        data = read_her_torch_experiment_data('yumi_constrained_v2')
        cache_result(result_id, data)
        
    print(data.keys())
    df = list(data.values())[0]
    success_rate = df['test_success_rate'].values
    steps_per_epoch = df['env_steps'][0]

    fig = plt.figure(figsize=(8,4))
    ax1 = fig.gca()
    ax1.plot(df['training_iteration'].values, success_rate)
    ax1.set_ylabel('Success rate')
    ax1.set_xlabel('Epoch ({} env. steps each)'.format(steps_per_epoch))

    ax2 = ax1.twiny()
    ax2.plot(df['total_time'].values, success_rate).pop(0).remove()

    def _format_date(x, pos=None):
        return str(timedelta(seconds=x))

    ax2.xaxis.set_major_formatter(ticker.FuncFormatter(_format_date))
    for tick in ax2.get_xticklabels():
        tick.set_rotation(45)
        
    ax1.grid()

_plot()
# plt.show()
plt.savefig(f'{FIGURES_DIR}/yumi_constr_sparse_her_torch.pdf', bbox_inches='tight')

'texgyreheros-regular.otf' can not be subsetted into a Type 3 font. The entire font will be embedded in the output.


dict_keys(['2019-05-10_00-30-08', '2019-05-09_10-29-40'])


'texgyreheros-regular.otf' can not be subsetted into a Type 3 font. The entire font will be embedded in the output.
